In [1]:
import equilibrate_phase_diagram
from thermoengine.const import units
import numpy as np

Tmin = 773
Tmax = 1273
nT = 10

Pmin = 0.5*units.GPA
Pmax = 2.5*units.GPA
nP = 10

T = np.linspace(Tmin,Tmax,nT)
P = np.linspace(Pmin,Pmax,nP)
name = "sammon-LC"

oxides_sammon={
    "SiO2"  :  53.3,
    "TiO2"  :  0.98,
    "Al2O3" :  17.2,
    "FeO"   :  7.12,
    "MnO"   :  0.16,
    "MgO"   :  7.63,
    "CaO"   : 10.15,
    "Na2O"  :  2.63,
    "K2O"   :  0.70,
    "P2O5"  :  0.16,
}

oxides = oxides_sammon
# Stixrude doesn't support P2O5, K2O, MnO, or TiO2
oxides["Na2O"] += oxides["K2O"]
oxides["Al2O3"] += oxides["P2O5"]
oxides["MgO"] += oxides["MnO"]
oxides["SiO2"] += oxides["TiO2"]
oxides["TiO2"] = 0
oxides["P2O5"] = 0
oxides["K2O"] = 0
oxides["MnO"] = 0

Ts, Ps, assemblages, system = equilibrate_phase_diagram.run(
    database="Stixrude",
    name=name,
    T=T,
    P=P,
    oxides=oxides
)

/opt/conda/lib/python3.8/site-packages/numdifftools/extrapolation.py:10: DeprecationWarning: Please use `convolve1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import convolve1d
/opt/conda/lib/python3.8/site-packages/numdifftools/multicomplex.py:35: DeprecationWarning: `finfo.machar` is deprecated (NumPy 1.22)
  _TINY = np.finfo(float).machar.tiny


ModuleNotFoundError: No module named 'py_tcg_slb_database'

In [ ]:
system.comp

'OxideWtComp(SiO2=53.3, TiO2=0.98, Al2O3=17.2, Fe2O3=0, Cr2O3=0, FeO=7.12, MnO=0.16, MgO=7.63, NiO=0, CoO=0, CaO=10.15, Na2O=2.63, K2O=0, P2O5=0, H2O=0, CO2=0)'

In [ ]:
system.comp
names = system.stable_assemblage.names
ms = system.stable_assemblage.amounts
ems = system.stable_assemblage.sample_endmem_comps

nameToM = {}
nameToEndmemberToC = {}

for i,name in enumerate(names):
    nameToM[name] = ms[i]
    #nameToEndmemberToC[name] = 


# write results to dict
# that can be dumped to json and read by TCG
results = {
    "T":T,
    "P":P,
    "comp":system.stable_assemblage,
    "mi0": "......"
}



In [ ]:
system.summary.report()

TypeError: 'list' object is not callable

In [ ]:
system.stable_assemblage.sample_endmem_comps

array([array([1.]), array([0.4, 0.4, 0.2, 0. , 0. ]),
       array([0.5, 0.2, 0.1, 0. , 0.2]), array([0.9, 0.1]),
       array([0.        , 0.25971214, 0.34028786, 0.        , 0.4       ])],
      dtype=object)

In [ ]:
system.stable_assemblage.elem_comps

,O,Na,Mg,Al,Si,Ca,Fe
Qz,0.666667,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000
Grt,0.600000,0.000000,0.060000,0.100000,0.150000,0.030000,0.060000
Cpx,0.600000,0.020000,0.070000,0.020000,0.200000,0.070000,0.020000
Fsp,0.615385,0.007692,0.000000,0.146154,0.161538,0.069231,0.000000
Cpx,0.600000,0.040000,0.068058,0.040000,0.200000,0.025971,0.025971
